# Universidad de O'Higgins

## Escuela de Ingeniería
## COM4402: Introducción a Inteligencia Artificial

### **Tarea 1: Clustering & Reducción de Dimensionalidad**

### Estudiante: Bastián Rubio Moya


El objetivo de esta tarea es utilizar distintos algoritmos de clustering y analizar su desempeño. Se utilizará la base de datos Mice Protein Expression Data Set, que es una base de datos tomadas del UC Irvine Machine Learning Repository. Contiene niveles de expresión de 77 proteínas medidas en fracciones nucleares de células en la corteza cerebral. Hay 38 ratones de control y 34 ratones trisómicos (síndrome de Down). Dado que algunos ratones fueron estimulados para aprender y/o inyectados con memantina, hay 8 grupos en total.

Se pide utilizar tres algoritmos de clustering en este trabajo: K-Means, DBSCAN y Clustering aglomerativo.

## En los Pasos 0.1 y 0.2, solo debe ejecutar las celdas respectivas

### Paso 0.1: Importación de librerías

In [ ]:
import pandas as pd
import numpy as np

from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA

# Para evitar problemas de versión con xlrd
!pip install --upgrade xlrd

### Paso 0.2: Definición de funciones auxiliares

In [ ]:
def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = kmeans.fit(data)
    fit_time = time() - t0
    results = [name, estimator.n_clusters, fit_time, estimator.inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator.labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(
            data,
            estimator.labels_,
            metric="euclidean",
            #sample_size=300, # No se usa, para que el calculo sea parejo respecto a bench_clustering_2
        )
    ]

    # Show the results
    formatter_result = (
        "{:9s}\t{:.0f}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}"
    )
    print(formatter_result.format(*results))

In [ ]:
def bench_clustering2(estim, name, data, labels, use_outliers= False):
    """Benchmark to evaluate the DBSCAN and AgglomerativeClustering initialization methods.

    Parameters
    ----------
    estim : DBSCAN or AGC instance
        A :class:`~sklearn.cluster.DBSCAN` or  `~sklearn.cluster.AgglomerativeClustering` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = estim.fit(data)
    fit_time = time() - t0


    e_labels = estimator.labels_

    # Ni DBSCAN ni Agglomerative poseen la variable inertia_., DBSCAN no posee la variable n_clusters
    try:
      n_clusters = estimator.n_clusters
      results = [name, n_clusters, fit_time, -1]
    except:
      # Es dbscan (falla en n_clusters)

      if use_outliers: # Se usan los outliers
        n_clusters = len(set(e_labels))
        results = [name, n_clusters, fit_time, -1]

      else: # No se usan los outliers
        n_clusters = len(set(e_labels))- (1 if -1 in e_labels else 0)
        results = [name, n_clusters, fit_time, -1]

        # Sólo se usan los datos que no contemplan outliers
        indexs = np.argwhere(e_labels!=-1)
        if len(indexs)>1:
          indexs = np.concatenate(indexs)
          data = data[indexs,]
          labels = labels[indexs,]
          e_labels = e_labels[indexs,]


    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    # Si hay más de un cluster
    if n_clusters>1:
      results += [m(labels, e_labels) for m in clustering_metrics]

      # The silhouette score requires the full dataset
      results += [
          metrics.silhouette_score(
              data,
              e_labels,
              metric="euclidean",
              #sample_size= None, # No se usa ya que puede dar errores si la muestra extraída solamente posee un solo label
          )
      ]
    else:
      results += [-1 for i in range(len(clustering_metrics)+1)]

    # Show the results
    formatter_result = (
        "{:9s}\t{:.0f}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}"
    )
    print(formatter_result.format(*results))

## De aquí en adelante empezará su implementación, la cuál será guiada

In [ ]:
#Conexión a drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

# Parte 1

Implemente un código que lea la base de datos "data_cortex_nuclear.csv" proporcionada en U-Campus. Se recomienda usar la biblioteca Pandas.

Se debe eliminar la columna BCL2_N, dado que contiene muchos valores NaN. Asimismo, las filas que contienen valores NaN deben ser eliminadas. Finalmente, para esta tarea, se recomienda no usar scale( ) sobre los datos.

In [ ]:
# Implemente en esta celda un codigo en Pandas que lea la base de datos "data_cortex_nuclear.csv". Hint: busque sobre pandas.ExcelFile

#Cargar data desde drive

data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/IA/Tarea 1/Data_Cortex_Nuclear.xls') #Modificar url en caso de ser necesario
data = data.drop(columns=['MouseID','BCL2_N', 'Genotype', 'Treatment', 'Behavior'])
data = data.dropna() #Se eliminan las filas con valores NaN

## Parte 2 & 3

Entrenar los siguientes algoritmos de clustering, con los parámetros especificados:
    
*   K-Means (con inicialización al azar), usando 8 clusters
*   K-Means++, usando 8 clusters
*   DBSCAN con épsilon por defecto
*   DBSCAN con épsilon 0.7
*   DBSCAN con épsilon 0.2
*   DBSCAN con épsilon por defecto, agregando outliers a cluster extra
*   DBSCAN con épsilon 0.7, agregando outliers a cluster extra
*   DBSCAN con épsilon 0.2, agregando outliers a cluster extra
*   Clustering aglomerativo, usando 8 clusters

Para el entrenamiento, use las funciones bench_k_means( ) y bench_clustering2( ), las cuales serán usadas para obtener un benchmark de K-Means, DBSCAN y Clustering Aglomerativo.

In [ ]:
#Paso previo cargamos "digits dataset" como lo indican en las instrucciones

labels = np.array(data['class'])
new_columns = []
for i in data.columns:
  if i == 'class':
    break
  new_columns.append(i)
data = np.array(data[new_columns])


In [ ]:
# Puede usar los metodos KMeans, DBSCAN, AgglomerativeClustering previamente cargados de la libreria sklearn.cluster

print(90*'_')
print('Nombre\t\tinit\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhoutte')

#K-Means (con inicialización al azar), usando 8 clusters
kmeans = KMeans(init= 'random', n_clusters=8, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="k-means_Random", data=data, labels=labels)

#K-Means++, usando 8 clusters
kmeansplus = KMeans(init= 'k-means++', n_clusters=8, n_init=4, random_state=0).fit(data)
bench_k_means(kmeans=kmeansplus, name="k-means_++", data=data, labels=labels)

#DBSCAN con épsilon por defecto
dbscan1 = DBSCAN().fit(data)
bench_clustering2(estim=dbscan1, name='DBSCAN Def', data= data, labels=labels, use_outliers= False)
#DBSCAN con épsilon 0.7
dbscan1 = DBSCAN(eps=0.7).fit(data)
bench_clustering2(estim=dbscan1, name='DBSCAN.7', data= data, labels=labels, use_outliers= False)
#DBSCAN con épsilon 0.2
dbscan1 = DBSCAN(eps=0.2).fit(data)
bench_clustering2(estim=dbscan1, name='DBSCAN.2', data= data, labels=labels, use_outliers= False)
#DBSCAN con épsilon por defecto, agregando outliers a cluster extra
dbscan1 = DBSCAN().fit(data)
bench_clustering2(estim=dbscan1, name='DBSCAN Def CE', data= data, labels=labels, use_outliers= True)
#DBSCAN con épsilon 0.7, agregando outliers a cluster extra
dbscan1 = DBSCAN(eps=0.7).fit(data)
bench_clustering2(estim=dbscan1, name='DBSCAN.7 CE', data= data, labels=labels, use_outliers= True)
#DBSCAN con épsilon 0.2, agregando outliers a cluster extra
dbscan1 = DBSCAN(eps=0.2).fit(data)
bench_clustering2(estim=dbscan1, name='DBSCAN.2 CE', data= data, labels=labels, use_outliers= True)
#Clustering aglomerativo, usando 8 clusters
agglo = AgglomerativeClustering(n_clusters=8).fit(data)
bench_clustering2(estim=agglo, name='Agglomerative', data= data, labels=labels, use_outliers= False)

print(90*'_')







__________________________________________________________________________________________
Nombre		init	time	inertia	homo	compl	v-meas	ARI	AMI	silhoutte
k-means_Random	8	0.033s	1127	0.280	0.282	0.281	0.130	0.266	0.209
k-means_++	8	0.214s	1124	0.310	0.311	0.310	0.148	0.296	0.210
DBSCAN Def	12	0.009s	-1	1.000	0.705	0.827	0.539	0.771	0.693
DBSCAN.7 	43	0.011s	-1	0.975	0.536	0.692	0.263	0.630	0.373
DBSCAN.2 	0	0.011s	-1	-1.000	-1.000	-1.000	-1.000	-1.000	-1.000
DBSCAN Def CE	13	0.011s	-1	0.107	0.365	0.165	0.002	0.113	-0.349
DBSCAN.7 CE	44	0.011s	-1	0.579	0.425	0.490	0.072	0.422	-0.006
DBSCAN.2 CE	1	0.006s	-1	-1.000	-1.000	-1.000	-1.000	-1.000	-1.000
Agglomerative	8	0.020s	-1	0.352	0.356	0.354	0.168	0.341	0.159
__________________________________________________________________________________________


## Parte 2 & 4

Reduzca la base de datos cargada y pre-procesada a 2 dimensiones, usando PCA

Entrenar los siguientes algoritmos de clustering, con los parámetros especificados:
    
*   K-Means (con inicialización al azar), usando 8 clusters
*   K-Means++, usando 8 clusters
*   DBSCAN con épsilon por defecto
*   DBSCAN con épsilon 0.7
*   DBSCAN con épsilon 0.2
*   DBSCAN con épsilon por defecto, agregando outliers a cluster extra
*   DBSCAN con épsilon 0.7, agregando outliers a cluster extra
*   DBSCAN con épsilon 0.2, agregando outliers a cluster extra
*   Clustering aglomerativo, usando 8 clusters

Para el entrenamiento, use las funciones bench_k_means( ) y bench_clustering2( ), las cuales serán usadas para obtener un benchmark de K-Means, DBSCAN y Clustering Aglomerativo.

In [ ]:
#Reducir datos con PCA a dos dimensiones
reduce_data = PCA(n_components=2).fit_transform(data)


In [ ]:
# Puede usar los metodos PCA de la libreria sklearn.decomposition y KMeans, DBSCAN, AgglomerativeClustering de la libreria sklearn.cluster

print(90*'_')
print('Nombre\t\tinit\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhoutte')

#K-Means (con inicialización al azar), usando 8 clusters
kmeans = KMeans(init= 'random', n_clusters=8).fit(reduce_data)
bench_k_means(kmeans=kmeans, name="k-means_Random", data=reduce_data, labels=labels)

#K-Means++, usando 8 clusters
kmeansplus = KMeans(init= 'k-means++', n_clusters=8).fit(reduce_data)
bench_k_means(kmeans=kmeansplus, name="k-means_++", data=reduce_data, labels=labels)

#DBSCAN con épsilon por defecto
dbscan1 = DBSCAN().fit(reduce_data)
bench_clustering2(estim=dbscan1, name='DBSCAN Def', data=reduce_data, labels=labels, use_outliers= False)
#DBSCAN con épsilon 0.7
dbscan1 = DBSCAN(eps=0.7).fit(reduce_data)
bench_clustering2(estim=dbscan1, name='DBSCAN.7', data=reduce_data, labels=labels, use_outliers= False)
#DBSCAN con épsilon 0.2
dbscan1 = DBSCAN(eps=0.2).fit(reduce_data)
bench_clustering2(estim=dbscan1, name='DBSCAN.2', data=reduce_data, labels=labels, use_outliers= False)
#DBSCAN con épsilon por defecto, agregando outliers a cluster extra
dbscan1 = DBSCAN().fit(reduce_data)
bench_clustering2(estim=dbscan1, name='DBSCAN Def CE', data=reduce_data, labels=labels, use_outliers= True)
#DBSCAN con épsilon 0.7, agregando outliers a cluster extra
dbscan1 = DBSCAN(eps=0.7).fit(reduce_data)
bench_clustering2(estim=dbscan1, name='DBSCAN.7 CE', data=reduce_data, labels=labels, use_outliers= True)
#DBSCAN con épsilon 0.2, agregando outliers a cluster extra
dbscan1 = DBSCAN(eps=0.2).fit(reduce_data)
bench_clustering2(estim=dbscan1, name='DBSCAN.2 CE', data=reduce_data, labels=labels, use_outliers= True)
#Clustering aglomerativo, usando 8 clusters
agglo = AgglomerativeClustering(n_clusters=8).fit(reduce_data)
bench_clustering2(estim=agglo, name='Agglomerative', data=reduce_data, labels=labels, use_outliers= False)


print(90*'_')











__________________________________________________________________________________________
Nombre		init	time	inertia	homo	compl	v-meas	ARI	AMI	silhoutte
k-means_Random	8	0.040s	347	0.209	0.212	0.211	0.096	0.194	0.366
k-means_++	8	0.033s	345	0.211	0.215	0.213	0.094	0.197	0.369
DBSCAN Def	1	0.006s	-1	-1.000	-1.000	-1.000	-1.000	-1.000	-1.000
DBSCAN.7 	1	0.007s	-1	-1.000	-1.000	-1.000	-1.000	-1.000	-1.000
DBSCAN.2 	20	0.005s	-1	0.284	0.283	0.284	0.068	0.212	0.083
DBSCAN Def CE	2	0.009s	-1	0.006	0.116	0.011	0.000	0.005	0.374
DBSCAN.7 CE	2	0.007s	-1	0.005	0.335	0.010	0.000	0.005	0.505
DBSCAN.2 CE	21	0.005s	-1	0.195	0.202	0.198	0.037	0.143	-0.205
Agglomerative	8	0.013s	-1	0.219	0.226	0.223	0.102	0.206	0.354
__________________________________________________________________________________________


##Outputs:

Nombre, N° Clusters, tiempo, Inercia, Homogeneidad, Completitud, Medida V, rand_Score, info_score, Puntuación de Silueta